In [10]:
import pandas as pd
import datetime

In [3]:
df = pd.read_csv("C:\\allcolumnsadded\\heater_1.csv")
df.columns

Index(['Unnamed: 0', 'Time', '101 <A LV Input> (VAC)',
       '102 <A HV Input> (VAC)', '103 <HTR1 Current> (AAC)',
       '104 <HTR1 Leakage> (MA)', '208 <Htr1 sn side 10 in> (C)',
       '209 <Htr1 sn side 5 in> (C)', '210 <Htr1 center> (C)',
       '211 <Htr1 5 in> (C)', '212 <Htr1 10 in> (C)', '901 <A HV Out> (VAC)',
       '902 <A LV Out> (VAC)', '903 <Htr1 Power> (WAT)', 'Sequence Number',
       'Status', 'Cycles_survived', 'Cycles_until_failure', 'Operation_time',
       'Time_until_failure', 'delta_HV', 'delta_LV', 'averagetemps'],
      dtype='object')

In [24]:
columnheaders = ['Unnamed', 'Time', 'LV Input (VAC)',
       'HV Input (VAC)', 'Current (AAC)',
       'Leakage (MA)', 'sn side 10 in (C)',
       'sn side 5 in (C)', 'center (C)',
       'side 5 in (C)', 'side 10 in (C)', 'HV Out (VAC)',
       'LV Out (VAC)', 'Power (WAT)', 'Sequencenumber',
       'Status', 'Cycles_survived', 'Cycles_until_failure', 'Operation_time',
       'Time_until_failure', 'delta_HV', 'delta_LV', 'averagetemps']
a = pd.read_csv("C:\\allcolumnsadded\\heater_1.csv")
a.columns.values[:] = columnheaders
sequencenumbers = ['sn1002','sn1003','sn2002','sn2003','sn3002','sn3003','sn5002','sn6002']
a = a.loc[a['Sequencenumber'].isin(sequencenumbers)]
for i in range(2,7):
    if i ==4:
        continue
    name = "C:\\allcolumnsadded\\heater_" + str(i) + ".csv"
    b = pd.read_csv(name)
    b.columns.values[:] =columnheaders
    df1 = b.loc[b['Sequencenumber'].isin(sequencenumbers)]
    a = pd.concat([a, df1], ignore_index=True)
outputname = "C:\\allcolumnsadded\\final.csv"
a.to_csv(outputname, index = False)

In [22]:
from datetime import timedelta, datetime
def operationtime(dataset):
    
    # Setup the dataset and extra lists
    times = dataset['Time']
    df = pd.DataFrame({'Time':times})
    time1 = pd.to_datetime(df['Time'])
    threshhold = datetime.strptime("2021/01/01 10:46:00", "%Y/%m/%d %H:%M:%S") - datetime.strptime("2021/01/01 10:44:00", "%Y/%m/%d %H:%M:%S")
    status = dataset['Status']
    operationtime = [time1[1]-time1[0]]
    
    #Add operationtime
    temp = time1[0]-time1[0]
    mostrecentworking = time1[0]
    for i in range(1,len(time1)):
        if status[i] == 'working':
            if status[i-1] == 'failed':
                temp = time1[0] - time1[0]
            delta = time1[i]-time1[i-1]
            if(delta<threshhold):
                temp = temp + delta
            operationtime.append(temp)
            mostrecentworking = temp
        else:
            operationtime.append(mostrecentworking)
    dataset["Operation_time"] = operationtime
    
    #Add Time until failure
    failuretime = []
    a = dataset['Time']
    b = dataset['Status']
    previousfail = 0
    time_until_failure = []
    for i in range(1, len(b)):
        if b[i]=='failed' and b[i-1] == 'working':
            for j in range(i-previousfail):
                time_until_failure.append((datetime.strptime(a[i-1],"%Y-%m-%d %H:%M:%S.%f")-datetime.strptime(a[j+previousfail],"%Y-%m-%d %H:%M:%S.%f")).total_seconds())
            time_until_failure.append((time1[0]-time1[0]).total_seconds())    
        elif b[i]=='failed':
            time_until_failure.append((time1[0]-time1[0]).total_seconds())
        elif b[i]=='working' and b[i-1] == 'failed':
            previousfail = i
        else:
            continue
    while len(time_until_failure) < len(a):
        time_until_failure.append(0)
    dataset["OperationTime_until_failure"] = time_until_failure

In [23]:
df = pd.read_csv("C:\\allcolumnsadded\\final.csv")
operationtime(df)
df = df[['LV Input (VAC)', 'HV Input (VAC)',
       'Current (AAC)', 'Leakage (MA)', 'sn side 10 in (C)',
       'sn side 5 in (C)', 'center (C)', 'side 5 in (C)',
       'side 10 in (C)', 'HV Out (VAC)', 'LV Out (VAC)', 'Power (WAT)', 'Status', 'averagetemps', 'OperationTime_until_failure']]
df.head()
df.loc[df['Status'] == 'working', 'Status'] = 0
df.loc[df['Status'] == 'failed', 'Status'] = 1

In [24]:
df.to_csv("C:\\allcolumnsadded\\fullyprocessedv2.csv", index = False)